#### LOAD & PROCESS DATA

In [ ]:
import pandas as pd
import numpy as np
from collections import deque

print("LOADING REAL BIBLE DATA (BradyStephenson/bible-data)")


base_url = "https://raw.githubusercontent.com/BradyStephenson/bible-data/main"


# BibleData-Person.csv and BibleData-PersonRelationship.csv are the real files
df_persons = pd.read_csv(f"{base_url}/BibleData-Person.csv")
df_rels    = pd.read_csv(f"{base_url}/BibleData-PersonRelationship.csv")

print(f"Loaded {len(df_persons)} people and {len(df_rels)} relationship records.")


LOADING REAL BIBLE DATA (BradyStephenson/bible-data)
Loaded 3009 people and 5450 relationship records.


In [ ]:
target_rels = ['father', 'mother', 'son', 'daughter']
df_family = df_rels[df_rels['relationship_type'].str.lower().isin(target_rels)].copy()

# Normalize direction so we always have Parent -> Child
# person_id_1 and person_id_2 are the endpoints.
df_family['parent_id'] = None
df_family['child_id']  = None

mask_parent = df_family['relationship_type'].str.lower().isin(['father', 'mother'])
mask_child  = df_family['relationship_type'].str.lower().isin(['son', 'daughter'])

# If relationship_type is father/mother: subject (person_id_1) is parent
df_family.loc[mask_parent, 'parent_id'] = df_family.loc[mask_parent, 'person_id_1']
df_family.loc[mask_parent, 'child_id']  = df_family.loc[mask_parent, 'person_id_2']

# If relationship_type is son/daughter: subject (person_id_1) is child -> flip
df_family.loc[mask_child, 'parent_id'] = df_family.loc[mask_child, 'person_id_2']
df_family.loc[mask_child, 'child_id']  = df_family.loc[mask_child, 'person_id_1']

# Drop any rows where we failed to assign parent/child (just in case)
df_family = df_family.dropna(subset=['parent_id', 'child_id'])

# 3. Create matrix index maps
valid_ids = set(df_family['parent_id']).union(set(df_family['child_id']))
sorted_ids = sorted(list(valid_ids))

id_to_idx = {real_id: i for i, real_id in enumerate(sorted_ids)}
idx_to_id = {i: real_id for i, real_id in enumerate(sorted_ids)}

In [11]:

N = len(sorted_ids)
Parent = np.zeros((N, N), dtype=int)

# 4. Fill adjacency matrix: Parent[parent_idx, child_idx] = 1
for _, row in df_family.iterrows():
    p_id = row['parent_id']
    c_id = row['child_id']
    p_idx = id_to_idx[p_id]
    c_idx = id_to_idx[c_id]
    Parent[p_idx, c_idx] = 1

# 5. Build name list aligned with our matrix indices
# In BibleData-Person.csv the key columns are usually: person_id, person_name
id_to_name_dict = dict(zip(df_persons['person_id'], df_persons['person_name']))
names = [id_to_name_dict.get(real_id, f"Unknown_{real_id}") for real_id in sorted_ids]

print(f"Matrix Construction Complete.")
print(f"Matrix Shape: {Parent.shape} (Nodes: {N})")


Matrix Construction Complete.
Matrix Shape: (1972, 1972) (Nodes: 1972)


#### TRANSITIVE CLOSURE ENGINE

In [ ]:

def print_relationships(matrix, title):
    """Just count number of 1s for large datasets."""
    count = int(np.sum(matrix))
    print(f"\n{title}: {count} relationships found")
    return count

print("TRANSITIVE CLOSURE: CONVERGENCE TRACE")

In [12]:

initial_count = print_relationships(Parent, "Direct parent relationships")

Ancestor = Parent.copy()
max_iterations = 100  # biblical lineages can be deep

print("CONVERGENCE ITERATIONS")



Direct parent relationships: 1727 relationships found
CONVERGENCE ITERATIONS


In [13]:
for iteration in range(max_iterations):
    # Rule: Ancestor(x,z) <- Ancestor(x,y), Parent(y,z)
    new_relations = np.einsum('xy,yz->xz', Ancestor, Parent)

    # Merge with existing (boolean OR)
    new_Ancestor = np.where((Ancestor + new_relations) > 0, 1, 0)

    # What’s actually new?
    newly_discovered = new_Ancestor - Ancestor
    new_count = int(np.sum(newly_discovered))

    if new_count == 0:
        print(f"✓ CONVERGED at Iteration {iteration + 1}")
        Ancestor = new_Ancestor
        break
    else:
        print(f"Iteration {iteration + 1}: Discovered {new_count} new ancestor links...")
        Ancestor = new_Ancestor
else:
    # If we exit without break
    print("\n Reached max_iterations without full convergence (consider increasing it).")

print("\nFINAL RESULTS")
final_count = print_relationships(Ancestor, "Complete ancestor relationships")
print(f"Total New Links discovered: {final_count - initial_count}")

Iteration 1: Discovered 1414 new ancestor links...
Iteration 2: Discovered 1412 new ancestor links...
Iteration 3: Discovered 1515 new ancestor links...
Iteration 4: Discovered 1536 new ancestor links...
Iteration 5: Discovered 1532 new ancestor links...
Iteration 6: Discovered 1442 new ancestor links...
Iteration 7: Discovered 1383 new ancestor links...
Iteration 8: Discovered 1321 new ancestor links...
Iteration 9: Discovered 1252 new ancestor links...
Iteration 10: Discovered 1185 new ancestor links...
Iteration 11: Discovered 1130 new ancestor links...
Iteration 12: Discovered 992 new ancestor links...
Iteration 13: Discovered 955 new ancestor links...
Iteration 14: Discovered 966 new ancestor links...
Iteration 15: Discovered 929 new ancestor links...
Iteration 16: Discovered 899 new ancestor links...
Iteration 17: Discovered 870 new ancestor links...
Iteration 18: Discovered 855 new ancestor links...
Iteration 19: Discovered 855 new ancestor links...
Iteration 20: Discovered 830 

#### VERIFICATION

In [14]:
def verify_transitive_closure(P, A):
    P = (P > 0).astype(int)
    A = (A > 0).astype(int)


    # 1) Every direct parent edge must appear in Ancestor
    print("[1/3] Checking: Parent ⊆ Ancestor")
    assert np.all((P == 0) | (A == 1)), "Missing a direct parent link in Ancestor"
    print("  ✓ PASS")

    # 2) Closure: A @ P adds nothing new
    print("[2/3] Checking: Closure property (A @ Parent adds nothing)")
    test_closure = (A @ P) > 0
    diff = (test_closure & (A == 0))
    assert not np.any(diff), "Closure not reached (A @ Parent found new edges)"
    print("  ✓ PASS")

    # 3) No cycles on diagonal (in ideal Bible lineages)
    print("[3/3] Checking: No cycles (diagonal = 0)")
    cycles = int(np.diag(A).sum())
    if cycles > 0:
        print(f"  ! NOTE: {cycles} self-loops found (data quirks / loops in raw dataset).")
    else:
        print("  ✓ PASS")

try:
    verify_transitive_closure(Parent, Ancestor)
    print("\n *** ALL CHECKS PASSED ***")
except AssertionError as e:
    print(f"\n --- Verification failed --- : {e}")


[1/3] Checking: Parent ⊆ Ancestor
  ✓ PASS
[2/3] Checking: Closure property (A @ Parent adds nothing)
  ✓ PASS
[3/3] Checking: No cycles (diagonal = 0)
  ✓ PASS

 *** ALL CHECKS PASSED ***


#### INTERPRETATION (Targeted)

In [16]:
def show_lineage(name_query, top_n=None):
    """
    Show lineage summary for a given person.

    - If top_n is None (default): only print total ancestors/descendants.
    - If top_n is an int: also print up to top_n descendants and ancestors,
      ordered by generation distance (closest first).

    Args:
        name_query (str): Substring to match in names (case-insensitive).
        top_n (int or None): Max number of names to print per side, or None for summary only.
    """

    def bfs_levels(adj_matrix, start_idx):
        """
        BFS over adj_matrix (N x N) starting at start_idx.
        Returns dict: node_idx -> distance_in_edges (0 for start_idx).
        """
        N = adj_matrix.shape[0]
        dist = {start_idx: 0}
        q = deque([start_idx])

        while q:
            u = q.popleft()
            neighbors = np.where(adj_matrix[u] == 1)[0]
            for v in neighbors:
                if v not in dist:
                    dist[v] = dist[u] + 1
                    q.append(v)
        return dist

    # 1. Find index by fuzzy name match
    try:
        idx = next(i for i, n in enumerate(names) if name_query.lower() in n.lower())
    except StopIteration:
        print(f"\nCould not find '{name_query}' in the dataset.")
        return

    name = names[idx]

    # 2. Totals using Ancestor matrix
    total_desc = np.where(Ancestor[idx] == 1)[0].size
    total_anc  = np.where(Ancestor[:, idx] == 1)[0].size

    print(f"\n[{name}]")
    print(f"  → Ancestor of {total_desc} people in total.")
    print(f"  ← Descended from {total_anc} people in total.")

    # If no top_n requested: stop here 
    if top_n is None:
        return

    # 3. If top_n is set: compute chronological lists via BFS on Parent
    #    Descendants (downward)
    desc_dist = bfs_levels(Parent, idx)
    desc_dist.pop(idx, None)
    sorted_desc = sorted(desc_dist.items(), key=lambda x: (x[1], names[x[0]]))
    if top_n > 0:
        sorted_desc = sorted_desc[:top_n]

    #    Ancestors (upward: use Parent.T)
    anc_dist = bfs_levels(Parent.T, idx)
    anc_dist.pop(idx, None)
    sorted_anc = sorted(anc_dist.items(), key=lambda x: (x[1], names[x[0]]))
    if top_n > 0:
        sorted_anc = sorted_anc[:top_n]

    if sorted_desc:
        print(f"Top {len(sorted_desc)} descendants by generation:")
        for node_idx, dist in sorted_desc:
            print(f"      gen {dist:2d}: {names[node_idx]}")
    else:
        print("No descendants in the dataset.")

    if sorted_anc:
        print(f"\n Top {len(sorted_anc)} ancestors by generation:")
        for node_idx, dist in sorted_anc:
            print(f"      gen {dist:2d}: {names[node_idx]}")
    else:
        print("\n No ancestors in the dataset.")


#### TESTING 

In [18]:
print("INTERPRETATION (Selected Bible Figures)")

show_lineage("Adam")

# Summary + top_n chronological descendants/ancestors
show_lineage("Adam", top_n=5)
show_lineage("Abram", top_n=10)


INTERPRETATION (Selected Bible Figures)

[Adam]
  → Ancestor of 821 people in total.
  ← Descended from 0 people in total.

[Adam]
  → Ancestor of 821 people in total.
  ← Descended from 0 people in total.
Top 5 descendants by generation:
      gen  1: Abel
      gen  1: Cain
      gen  1: Seth
      gen  2: Enoch
      gen  2: Enosh

 No ancestors in the dataset.

[Abram]
  → Ancestor of 698 people in total.
  ← Descended from 20 people in total.
Top 10 descendants by generation:
      gen  1: Isaac
      gen  1: Ishbak
      gen  1: Ishmael
      gen  1: Jokshan
      gen  1: Medan
      gen  1: Midian
      gen  1: Shuah
      gen  1: Zimran
      gen  2: Abida
      gen  2: Adbeel

 Top 10 ancestors by generation:
      gen  1: Terah
      gen  2: Nahor
      gen  3: Serug
      gen  4: Reu
      gen  5: Peleg
      gen  6: Eber
      gen  7: Shelah
      gen  8: Arpachshad
      gen  9: Shem
      gen 10: Noah


In [3]:
N = len(sorted_ids)
Parent = np.zeros((N, N), dtype=int)

# 4. Fill adjacency matrix: Parent[parent_idx, child_idx] = 1
for _, row in df_family.iterrows():
    p_id = row['parent_id']
    c_id = row['child_id']
    p_idx = id_to_idx[p_id]
    c_idx = id_to_idx[c_id]
    Parent[p_idx, c_idx] = 1

# 5. Build name list aligned with our matrix indices
# In BibleData-Person.csv the key columns are usually: person_id, person_name
id_to_name_dict = dict(zip(df_persons['person_id'], df_persons['person_name']))
names = [id_to_name_dict.get(real_id, f"Unknown_{real_id}") for real_id in sorted_ids]

print(f"✓ Matrix Construction Complete.")
print(f"✓ Matrix Shape: {Parent.shape} (Nodes: {N})")
print("="*70)

✓ Matrix Construction Complete.
✓ Matrix Shape: (1972, 1972) (Nodes: 1972)


## Transitive closure engine 

In [ ]:
def print_relationships(matrix, title):
    """Just count number of 1s for large datasets."""
    count = int(np.sum(matrix))
    print(f"\n{title}: {count} relationships found")
    return count

print("TRANSITIVE CLOSURE: CONVERGENCE TRACE")
print("="*70)

initial_count = print_relationships(Parent, "Direct parent relationships")

Ancestor = Parent.copy()
max_iterations = 100  # biblical lineages can be deep

print("\n" + "="*70)
print("CONVERGENCE ITERATIONS")
print("="*70)

for iteration in range(max_iterations):
    # Rule: Ancestor(x,z) ← Ancestor(x,y), Parent(y,z)
    new_relations = np.einsum('xy,yz->xz', Ancestor, Parent)

    # Merge with existing (boolean OR)
    new_Ancestor = np.where((Ancestor + new_relations) > 0, 1, 0)

    # What’s actually new?
    newly_discovered = new_Ancestor - Ancestor
    new_count = int(np.sum(newly_discovered))

    if new_count == 0:
        print(f"\n{'='*35}")
        print(f"✓ CONVERGED at Iteration {iteration + 1}")
        print(f"{'='*35}")
        Ancestor = new_Ancestor
        break
    else:
        print(f"Iteration {iteration + 1}: Discovered {new_count} new ancestor links...")
        Ancestor = new_Ancestor
else:
    # If we exit without break
    print("\n⚠ Reached max_iterations without full convergence (consider increasing it).")

print("\nFINAL RESULTS")
final_count = print_relationships(Ancestor, "Complete ancestor relationships")
print(f"Total New Links discovered: {final_count - initial_count}")

TRANSITIVE CLOSURE: CONVERGENCE TRACE

Direct parent relationships: 1727 relationships found

CONVERGENCE ITERATIONS
Iteration 1: Discovered 1414 new ancestor links...
Iteration 2: Discovered 1412 new ancestor links...
Iteration 3: Discovered 1515 new ancestor links...
Iteration 4: Discovered 1536 new ancestor links...
Iteration 5: Discovered 1532 new ancestor links...
Iteration 6: Discovered 1442 new ancestor links...
Iteration 7: Discovered 1383 new ancestor links...
Iteration 8: Discovered 1321 new ancestor links...
Iteration 9: Discovered 1252 new ancestor links...
Iteration 10: Discovered 1185 new ancestor links...
Iteration 11: Discovered 1130 new ancestor links...
Iteration 12: Discovered 992 new ancestor links...
Iteration 13: Discovered 955 new ancestor links...
Iteration 14: Discovered 966 new ancestor links...
Iteration 15: Discovered 929 new ancestor links...
Iteration 16: Discovered 899 new ancestor links...
Iteration 17: Discovered 870 new ancestor links...
Iteration 18: 

## Verification

In [5]:
def verify_transitive_closure(P, A):
    P = (P > 0).astype(int)
    A = (A > 0).astype(int)

    print("\n" + "="*70)
    print("VERIFICATION")
    print("="*70)

    # 1) Every direct parent edge must appear in Ancestor
    print("[1/3] Checking: Parent ⊆ Ancestor")
    assert np.all((P == 0) | (A == 1)), "Missing a direct parent link in Ancestor"
    print("  ✓ PASS")

    # 2) Closure: A @ P adds nothing new
    print("[2/3] Checking: Closure property (A @ Parent adds nothing)")
    test_closure = (A @ P) > 0
    diff = (test_closure & (A == 0))
    assert not np.any(diff), "Closure not reached (A @ Parent found new edges)"
    print("  ✓ PASS")

    # 3) No cycles on diagonal (in ideal Bible lineages)
    print("[3/3] Checking: No cycles (diagonal = 0)")
    cycles = int(np.diag(A).sum())
    if cycles > 0:
        print(f"  ! NOTE: {cycles} self-loops found (data quirks / loops in raw dataset).")
    else:
        print("  ✓ PASS")

try:
    verify_transitive_closure(Parent, Ancestor)
    print("\n✓✓✓ ALL CHECKS PASSED ✓✓✓")
except AssertionError as e:
    print(f"\n✗ Verification failed: {e}")



VERIFICATION
[1/3] Checking: Parent ⊆ Ancestor
  ✓ PASS
[2/3] Checking: Closure property (A @ Parent adds nothing)
  ✓ PASS
[3/3] Checking: No cycles (diagonal = 0)
  ✓ PASS

✓✓✓ ALL CHECKS PASSED ✓✓✓


## Verification

In [6]:
from collections import deque

def show_lineage(name_query, top_n=None):
    """
    Show lineage summary for a given person.

    - If top_n is None (default): only print total ancestors/descendants.
    - If top_n is an int: also print up to top_n descendants and ancestors,
      ordered by generation distance (closest first).

    Args:
        name_query (str): Substring to match in names (case-insensitive).
        top_n (int or None): Max number of names to print per side, or None for summary only.
    """

    def bfs_levels(adj_matrix, start_idx):
        """
        BFS over adj_matrix (N x N) starting at start_idx.
        Returns dict: node_idx -> distance_in_edges (0 for start_idx).
        """
        N = adj_matrix.shape[0]
        dist = {start_idx: 0}
        q = deque([start_idx])

        while q:
            u = q.popleft()
            neighbors = np.where(adj_matrix[u] == 1)[0]
            for v in neighbors:
                if v not in dist:
                    dist[v] = dist[u] + 1
                    q.append(v)
        return dist

    # 1. Find index by fuzzy name match
    try:
        idx = next(i for i, n in enumerate(names) if name_query.lower() in n.lower())
    except StopIteration:
        print(f"\nCould not find '{name_query}' in the dataset.")
        return

    name = names[idx]

    # 2. Totals using Ancestor matrix
    total_desc = np.where(Ancestor[idx] == 1)[0].size
    total_anc  = np.where(Ancestor[:, idx] == 1)[0].size

    print(f"\n[{name}]")
    print(f"  → Ancestor of {total_desc} people in total.")
    print(f"  ← Descended from {total_anc} people in total.")

    # If no top_n requested: stop here (summary only)
    if top_n is None:
        return

    # 3. If top_n is set: compute chronological lists via BFS on Parent
    #    Descendants (downward)
    desc_dist = bfs_levels(Parent, idx)
    desc_dist.pop(idx, None)
    sorted_desc = sorted(desc_dist.items(), key=lambda x: (x[1], names[x[0]]))
    if top_n > 0:
        sorted_desc = sorted_desc[:top_n]

    #    Ancestors (upward: use Parent.T)
    anc_dist = bfs_levels(Parent.T, idx)
    anc_dist.pop(idx, None)
    sorted_anc = sorted(anc_dist.items(), key=lambda x: (x[1], names[x[0]]))
    if top_n > 0:
        sorted_anc = sorted_anc[:top_n]

    # 4. Pretty-print details
    if sorted_desc:
        print(f"  → Top {len(sorted_desc)} descendants by generation:")
        for node_idx, dist in sorted_desc:
            print(f"      gen {dist:2d}: {names[node_idx]}")
    else:
        print("  → No descendants in the dataset.")

    if sorted_anc:
        print(f"\n  ← Top {len(sorted_anc)} ancestors by generation:")
        for node_idx, dist in sorted_anc:
            print(f"      gen {dist:2d}: {names[node_idx]}")
    else:
        print("\n  ← No ancestors in the dataset.")

In [8]:
show_lineage("Adam")



[Adam]
  → Ancestor of 821 people in total.
  ← Descended from 0 people in total.


In [9]:
show_lineage("Adam", top_n=5)
show_lineage("Abram", top_n=10)


[Adam]
  → Ancestor of 821 people in total.
  ← Descended from 0 people in total.
  → Top 5 descendants by generation:
      gen  1: Abel
      gen  1: Cain
      gen  1: Seth
      gen  2: Enoch
      gen  2: Enosh

  ← No ancestors in the dataset.

[Abram]
  → Ancestor of 698 people in total.
  ← Descended from 20 people in total.
  → Top 10 descendants by generation:
      gen  1: Isaac
      gen  1: Ishbak
      gen  1: Ishmael
      gen  1: Jokshan
      gen  1: Medan
      gen  1: Midian
      gen  1: Shuah
      gen  1: Zimran
      gen  2: Abida
      gen  2: Adbeel

  ← Top 10 ancestors by generation:
      gen  1: Terah
      gen  2: Nahor
      gen  3: Serug
      gen  4: Reu
      gen  5: Peleg
      gen  6: Eber
      gen  7: Shelah
      gen  8: Arpachshad
      gen  9: Shem
      gen 10: Noah
